In [1]:
import pandas as pd
import numpy as np
import pickle
import time

In [2]:
# Work in progress, variable names have to be added below

train_persistence_diagrams_label_1 = np.load("Embeddings_and_Persistence_Diagrams/Train_PD1.npy", allow_pickle=True)
test_persistence_diagrams_label_1 = np.load("Embeddings_and_Persistence_Diagrams/Test_PD1.npy", allow_pickle=True)

train_persistence_diagrams_label_3 = np.load("Embeddings_and_Persistence_Diagrams/Train_PD3.npy", allow_pickle=True)
test_persistence_diagrams_label_3 = np.load("Embeddings_and_Persistence_Diagrams/Test_PD3.npy", allow_pickle=True)

train_persistence_diagrams_label_5 = np.load("Embeddings_and_Persistence_Diagrams/Train_PD5.npy", allow_pickle=True)
test_persistence_diagrams_label_5 = np.load("Embeddings_and_Persistence_Diagrams/Test_PD5.npy", allow_pickle=True)

train_persistence_diagrams_label_7 = np.load("Embeddings_and_Persistence_Diagrams/Train_PD7.npy", allow_pickle=True)
test_persistence_diagrams_label_7 = np.load("Embeddings_and_Persistence_Diagrams/Test_PD7.npy", allow_pickle=True)

In [10]:
# Adcock-Carlson coordinates

In [3]:
def carlsson_coordinates(X_train, X_test):
    n = len(X_train)
    X_train_features_cc1 = np.zeros(n)
    X_train_features_cc2 = np.zeros(n)
    X_train_features_cc3 = np.zeros(n)
    X_train_features_cc4 = np.zeros(n)
    start = time.time()
    ymax = 0
    for i in range(0,n):
        if len(X_train[i])>0:
            b = np.max(X_train[i][:,1])
        else:
            b = ymax
        if ymax < b:
            ymax = b
        else:
            ymax = ymax
    print(ymax)
    for i in range(0,n):
        if len(X_train[i])>0:
            x = X_train[i][:,0]
            y = X_train[i][:,1]
            X_train_features_cc1[i] = sum(x*(y-x))
            X_train_features_cc2[i] = sum((ymax - y)*(y-x))
            X_train_features_cc3[i] = sum(x**2*(y-x)**4)
            X_train_features_cc4[i] = sum((ymax-y)**2*(y-x)**4)
        else:
            X_train_features_cc1[i] = 0
            X_train_features_cc2[i] = 0
            X_train_features_cc3[i] = 0
            X_train_features_cc4[i] = 0

    n = len(X_test)
    X_test_features_cc1 = np.zeros(n)
    X_test_features_cc2 = np.zeros(n)
    X_test_features_cc3 = np.zeros(n)
    X_test_features_cc4 = np.zeros(n)
    ymax = 0
    for i in range(0,n):
        if len(X_test[i])>0:
            b = np.max(X_test[i][:,1])
        else:
            b = ymax
        if ymax < b:
            ymax = b
        else:
            ymax = ymax
    for i in range(0,n):
        if len(X_test[i])>0:
            x = X_test[i][:,0]
            y = X_test[i][:,1]
            X_test_features_cc1[i] = sum(x*(y-x))
            X_test_features_cc2[i] = sum((ymax - y)*(y-x))
            X_test_features_cc3[i] = sum(x**2*(y-x)**4)
            X_test_features_cc4[i] = sum((ymax-y)**2*(y-x)**4)
        else:
            X_test_features_cc1[i] = 0
            X_test_features_cc2[i] = 0
            X_test_features_cc3[i] = 0
            X_test_features_cc4[i] = 0
            
    return X_train_features_cc1, X_train_features_cc2, X_train_features_cc3, X_train_features_cc4, X_test_features_cc1, X_test_features_cc2, X_test_features_cc3, X_test_features_cc4

In [4]:
def create_dataframe(X_features_cc1, X_features_cc2, X_features_cc3, X_features_cc4, label):
    features_df = pd.DataFrame()

    features_df["AC_coordinate_cc1"] = X_features_cc1
    features_df["AC_coordinate_cc2"] = X_features_cc2
    features_df["AC_coordinate_cc3"] = X_features_cc3
    features_df["AC_coordinate_cc4"] = X_features_cc4
    features_df["Label"] = label

    return features_df

In [5]:
# LABEL 1

# Adcock Carlson coordinates 
train_carlson_coord_1, train_carlson_coord_2, train_carlson_coord_3, train_carlson_coord_4, test_carlson_coord_1, test_carlson_coord_2, test_carlson_coord_3, test_carlson_coord_4 = carlsson_coordinates(train_persistence_diagrams_label_1, test_persistence_diagrams_label_1)

# Create dataframe
train_features_df1 = create_dataframe(train_carlson_coord_1, train_carlson_coord_2, train_carlson_coord_3, train_carlson_coord_4, 1)
test_features_df1 = create_dataframe(test_carlson_coord_1, test_carlson_coord_2, test_carlson_coord_3, test_carlson_coord_4, 1)

0.00509025389328599


In [6]:
# LABEL 3

# Adcock Carlson coordinates 
train_carlson_coord_1, train_carlson_coord_2, train_carlson_coord_3, train_carlson_coord_4, test_carlson_coord_1, test_carlson_coord_2, test_carlson_coord_3, test_carlson_coord_4 = carlsson_coordinates(train_persistence_diagrams_label_3, test_persistence_diagrams_label_3)

# Create dataframe
train_features_df3 = create_dataframe(train_carlson_coord_1, train_carlson_coord_2, train_carlson_coord_3, train_carlson_coord_4, 3)
test_features_df3 = create_dataframe(test_carlson_coord_1, test_carlson_coord_2, test_carlson_coord_3, test_carlson_coord_4, 3)

0.0013039779150858521


In [7]:
# LABEL 5

# Adcock Carlson coordinates 
train_carlson_coord_1, train_carlson_coord_2, train_carlson_coord_3, train_carlson_coord_4, test_carlson_coord_1, test_carlson_coord_2, test_carlson_coord_3, test_carlson_coord_4 = carlsson_coordinates(train_persistence_diagrams_label_5, test_persistence_diagrams_label_5)

# Create dataframe
train_features_df5 = create_dataframe(train_carlson_coord_1, train_carlson_coord_2, train_carlson_coord_3, train_carlson_coord_4, 5)
test_features_df5 = create_dataframe(test_carlson_coord_1, test_carlson_coord_2, test_carlson_coord_3, test_carlson_coord_4, 5)

0.0007453467696905136


In [8]:
# LABEL 7

# Adcock Carlson coordinates 
train_carlson_coord_1, train_carlson_coord_2, train_carlson_coord_3, train_carlson_coord_4, test_carlson_coord_1, test_carlson_coord_2, test_carlson_coord_3, test_carlson_coord_4 = carlsson_coordinates(train_persistence_diagrams_label_7, test_persistence_diagrams_label_7)

# Create dataframe
train_features_df7 = create_dataframe(train_carlson_coord_1, train_carlson_coord_2, train_carlson_coord_3, train_carlson_coord_4, 7)
test_features_df7 = create_dataframe(test_carlson_coord_1, test_carlson_coord_2, test_carlson_coord_3, test_carlson_coord_4, 7)

0.010027261450886726


In [9]:
# Concatenate and save dataframes

train_feature_df = pd.concat([train_features_df1, train_features_df3, train_features_df5, train_features_df7], ignore_index=True)
test_feature_df = pd.concat([test_features_df1, test_features_df3, test_features_df5, test_features_df7], ignore_index=True)

train_feature_df.to_csv("Features/Train_Advanced_Features.csv")
test_feature_df.to_csv("Features/Test_Advanced_Features.csv")